In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import lr_scheduler

import torchvision
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

import random as rand
from random import *
import os
import cv2
import copy
import time
from PIL import Image

In [ ]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

data_dir = "../Data_Set/Labeld_Crop_Data/"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
class_names = trDsets['train'].classes

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [ ]:
class customResNet:
    def __init__(self, num, classes, loss, optimizer):
        if num == 18:
            self.model = models.resnet18(pretrained=False)
        elif num == 34:
            self.model = models.resnet32(pretrained=False)
        elif num == 50:
            self.model = models.resnet50(pretrained=False)
        elif num == 101:
            self.model = models.resnet101(pretrained=False)
        elif num == 152:
            self.model = models.resnet152(pretrained=False)
        
        self.num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(self.num_ftrs, len(classes))
        self.model = self.model.to(device)
        
        
        # Loss Function
        if loss == 'cross_entropy':
            self.criterion = nn.CrossEntropyLoss()
        elif loss == '':
            self.criterion = nn.
        elif loss == '':
            self.criterion = nn.CrossEntropyLoss()
        elif loss == '':
            self.criterion = nn.CrossEntropyLoss()
        
        # Optimizer
        if optimizer == 'SGD':
            self.optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        elif optimizer == 'Adam':
            self.optimizer_ft = optim.Adam()
        elif optimizer == 'RMSprop':
            self.optimizer_ft = optim.RMSprop()
        elif optimizer == 'Adagrad':
            self.optimizer_ft = optim.Adagrad()
        elif optimizer == 'Adadelta':
            self.optimizer_ft = optim.Adadelta()
        
        # LR_Scheduler
        self.lr_scheduler = lr_scheduler.ReduceLROnPlateau(self.optimizer_ft, factor=0.1, patience=11)

    def __getitem__(self):
        return self.model, self.criterion, self.optimizer_ft, self.lr_scheduler
        
def train_model(model, criterion, optimizer, scheduler, num_epochs=8):

    global_info = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=11, verbose=True)
    for epoch in range(num_epochs):
        local_info = []
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

                if epoch > 0:
                    scheduler.step(val_loss)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in trLoaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / trDsets_sizes[phase]
            if phase == 'val':
                val_loss = running_loss / trDsets_sizes['val']
            epoch_acc = running_corrects.double() / trDsets_sizes[phase]

            if phase == 'train':
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)
            else:
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        lr_get = get_lr(optimizer)
        print("Current learning rate : {:.8f}".format(lr_get))
        global_info.append(local_info)

        if phase =='val':
            early_stopping(epoch_loss, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
    

In [ ]:
num = 
classes = 
loss = 
optimizer = 

model, criterion, optimizer_ft, lr_scheduler = customResNet()

print(model)

In [ ]:
torch.cuda.empty_cache()
model_ft = train_model(model, criterion, optimizer_ft, lr_scheduler, num_epochs=100, device=device)

In [ ]:
torch.save(model_ft, f'./save/lotte_model_resnet{num}_{loss}.pt')